In [80]:
import csv
import pandas as pd
import geopandas as gpd
import pydeck as pdk
shapefile = '/Users/maxwell/Desktop/Trnasition of World Population/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp'
shapefile

'/Users/maxwell/Desktop/Trnasition of World Population/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp'

In [81]:
gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
gdf.columns = ['country', 'country_code', 'geometry']
gdf.head()

,country,country_code,geometry
0,Indonesia,IDN,"MULTIPOLYGON (((117.70361 4.16341, 117.70361 4..."
1,Malaysia,MYS,"MULTIPOLYGON (((117.70361 4.16341, 117.69711 4..."
2,Chile,CHL,"MULTIPOLYGON (((-69.51009 -17.50659, -69.50611..."
3,Bolivia,BOL,"POLYGON ((-69.51009 -17.50659, -69.51009 -17.5..."
4,Peru,PER,"MULTIPOLYGON (((-69.51009 -17.50659, -69.63832..."


In [82]:
print(gdf[gdf['country'] == 'Antarctica'])
gdf = gdf.drop(gdf.index[159])

        country country_code  \
172  Antarctica          ATA   

                                              geometry  
172  MULTIPOLYGON (((-51.73064 -82.06256, -51.24690...  


In [83]:
print(gdf[gdf['country'] == 'Antarctica'])

        country country_code  \
172  Antarctica          ATA   

                                              geometry  
172  MULTIPOLYGON (((-51.73064 -82.06256, -51.24690...  


In [84]:
import pandas as pd

datafile = 'projected-population-by-country.csv'

df = pd.read_csv(datafile, names = ['entity', 'code', 'year', 'population'], skiprows = 1)
df.head()

,entity,code,year,population
0,Afghanistan,AFG,1800,3280000
1,Afghanistan,AFG,1801,3280000
2,Afghanistan,AFG,1802,3280000
3,Afghanistan,AFG,1803,3280000
4,Afghanistan,AFG,1804,3280000


In [85]:
#df_2016 = df[df['year'] == 2016]
#Perform left merge to preserve every row in gdf.
merged = gdf.merge(df, left_on = 'country_code', right_on = 'code', how = 'left')

#year before 2021
merged = merged.loc[merged["year"] == '2021', :] 

#Replace NaN values to string 'No data'.
merged.fillna('No data', inplace = True)

#year 값을 object -> value 로 변환 
merged['year'] = merged['year'].astype(int)

# Column country, entity, code 삭제 
merged.drop(["country", 'entity', 'code'], axis='columns', inplace=True)

merged['population'] = merged['population'].astype(float)
#merged['geometry'] = merged['geometry'].astype(object)
merged

,country_code,geometry,year,population
221,IDN,"MULTIPOLYGON (((117.70361 4.16341, 117.70361 4...",2021,276362000.0
522,MYS,"MULTIPOLYGON (((117.70361 4.16341, 117.69711 4...",2021,32776000.0
823,CHL,"MULTIPOLYGON (((-69.51009 -17.50659, -69.50611...",2021,19212000.0
1124,BOL,"POLYGON ((-69.51009 -17.50659, -69.51009 -17.5...",2021,11833000.0
1425,PER,"MULTIPOLYGON (((-69.51009 -17.50659, -69.63832...",2021,33359000.0
...,...,...,...,...
61813,PLW,"MULTIPOLYGON (((134.27149 7.07453, 134.27931 7...",2021,18000.0
61964,GUM,"POLYGON ((144.88640 13.64020, 144.89666 13.617...",2021,170000.0
62115,MNP,"MULTIPOLYGON (((145.20574 14.18138, 145.25245 ...",2021,58000.0
62416,BHR,"POLYGON ((50.55161 26.19424, 50.59474 26.16031...",2021,1748000.0


In [86]:
def multipolygon_to_coordinates(x):
    lon, lat = x[0].exterior.xy
    return [[x, y] for x, y in zip(lon, lat)]

merged['coordinates'] = merged['geometry'].apply(multipolygon_to_coordinates)
del merged['geometry']

TypeError: 'Polygon' object is not subscriptable

In [87]:
#인구를 0 ~ 1 사이의 값으로 변환시킨다. 
merged['normal'] = merged['population'] / merged['population'].max()

In [88]:
import json

#Read data to json
merged_json = json.loads(merged.to_json())

#Convert to str like object
#json_data = json.dumps(merged_json) -> 안됨

#json_data

In [89]:
geojson = pdk.Layer(
    'GeoJsonLayer',
    merged_json ,
    opacity=0.8,
    stroked=False,
    filled=True,
    extruded=True,
    wireframe=True,
    get_elevation='normal * 100',
    get_fill_color='[ 255,  normal, 255]',
    #get_line_color=[255, 255, 255],
    pickable=True
)

INITIAL_VIEW_STATE = pdk.ViewState(
  latitude=49.254,
  longitude=-123.13,
  zoom=1,
  max_zoom=16,
  pitch=45,
  bearing=0
)

r = pdk.Deck(
    layers=[geojson],
    initial_view_state=INITIAL_VIEW_STATE)

r.show()

DeckGLWidget(json_input='{"initialViewState": {"bearing": 0, "latitude": 49.254, "longitude": -123.13, "maxZoo…

In [72]:
DATA_URL = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/examples/geojson/vancouver-blocks.json"
LAND_COVER = [[[-123.0, 49.196], [-123.0, 49.324], [-123.306, 49.324], [-123.306, 49.196]]]


In [73]:
INITIAL_VIEW_STATE = pdk.ViewState(
  latitude=49.254,
  longitude=-123.13,
  zoom=11,
  max_zoom=16,
  pitch=45,
  bearing=0
)

In [78]:
geojson = pdk.Layer(
    'GeoJsonLayer',
    DATA_URL,
    opacity=0.8,
    stroked=False,
    filled=True,
    #extruded=False,
    wireframe=True,
    #get_elevation='properties.valuePerSqm / 20',
    get_fill_color='[255, properties.growth * 255, 255]',
    get_line_color=[255, 255, 255],
    pickable=True
)


In [79]:
r = pdk.Deck(
    layers=[geojson],
    initial_view_state=INITIAL_VIEW_STATE)

r.show()

DeckGLWidget(json_input='{"initialViewState": {"bearing": 0, "latitude": 49.254, "longitude": -123.13, "maxZoo…